In [2]:
import random
import numpy as np
import os
import torch

# GPU가 인식되면 GPU사용 아니면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random_seed = 42

# 고정된 랜덤값을 생성하게 해줌
torch.manual_seed(random_seed)

# Random 값을 GPU에서 생성할 때를 위한 설정
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Randomness(Random값)를 제어하기 위한 CUDA 설정
# 이걸 설정하면 연산 속도가 느려질 수도 있음.
# 모델 배포 직전에 사용 추천
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# numpy에서 Random값을 고정 시켜줌.
np.random.seed(random_seed)
random.seed(random_seed)

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
NVIDIA GeForce RTX 2070 Super


In [3]:
# 여러가지 torch model을 불러올 수있는 패키지
import timm
# timm.list_models("vit*")

In [4]:
# model_name="efficientnet_b0" # batch 32
# model_name="swsl_resnext50_32x4d" # batch 32
# model_name = "eca_nfnet_l0" # batch 32
model_name = "vit_base_patch16_224"
epoch_size = 20
batch_size = 16

learning_rate = 1e-4
early_stop = 5
k_fold_num = 5

In [5]:
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

from sklearn.metrics import f1_score

# Train 

In [6]:
def train(data_loader) :
    model = timm.create_model(model_name, pretrained=True, num_classes=7).to(device=device)
    #     print(model)
    
    # class 개수 별로 가중치를 따로줌
    class_num = [329, 205, 235, 134, 151, 245, 399]
    class_weight = torch.tensor(np.max(class_num)/class_num).to(device=device,dtype=torch.float)
    # tensor([1.2128, 1.9463, 1.6979, 2.9776, 2.6424, 1.6286, 1.0000])

    # 위에서 구한 weight값(tensor)를 loss function에 넣어줌.
    criterion = nn.CrossEntropyLoss(weight=class_weight)

    # fc layer를 model에서 분리해서 각각의 weight값을 optimizer에 넣어줌
    # 각 weight값에 다른 learning rate를 적용해줌
    # model마다 fc layer의 이름이나 존재여부가 다를 수 있음.
    # efficientNet같은 경우에는 이름이 classifier 였고 분리가 안됐음.
    # n = name, m = weight
#     feature_extractor = [m for n, m in model.named_parameters() if "fc" not in n]
#     feature_extractor = [m for n, m in model.named_parameters()]
#     classifier = [p for p in model.fc.parameters()]
#     classifier = [p for p in model.classifier.parameters()]
#     params = [
#         {"params":feature_extractor, "lr":learning_rate * 0.5},
#         {"params":classifier, "lr":learning_rate}
#     ]
#     params = [
#             {"params":feature_extractor, "lr":learning_rate * 0.5}
#         ]


    # oprimizer 선언
    optimizer = AdamW(params=model.parameters(), lr=learning_rate)

    # training scheduler 정의.
    # CosineAnnealing Scheduler 적용  => CosineAnnealing Scheduler?>
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

    result = {
        "train_loss" : [],
        "valid_loss" : [],
        "valid_acc" : [],
        "valid_f1" : [],
    }

    # training에 필요한 dataset loader 정의
    train_loader = data_loader["train_loader"]
    valid_loader = data_loader["valid_loader"]

    best_model_state = None
    best_f1 = 0
    early_stop_count = 0

    # training 시작
    for epoch_idx in range(1, epoch_size + 1) :
        # model을 훈련 모드로 변경.
        # dropout, BatchNorm 등이 training에 맞춰서 작동하게 만들어줌.
        model.train()

        iter_train_loss = []
        iter_valid_loss = []
        iter_valid_acc = []
        iter_valid_f1 = []
        

        # 1 Epoch에 포함된 batch size만큼 iteration 진행
        for iter_idx, (train_imgs, train_labels) in enumerate(train_loader, 1) :
            # train dataset에서 가져온 data를 gpu 연산이 되도록 해줌.
            train_imgs, train_labels = train_imgs.to(device=device, dtype=torch.float), train_labels.to(device)
            
            # loss.backward 를 진행 할 때 gradient를 매번 더해주는 형식으로 진행됨.
            # 그래서 1 iter가 진행되면 gradient를 0으로 초기화 해줘야 
            # 다음 iter에서 이전에 받아온 gradient가 영향을 안 미치고 정상적인 방향으로 학습이 진행됨.
            optimizer.zero_grad()

            # model inference
            # model의 fc layer 이후에 softmax 나 sigmoid가 적용되어 있지 않아서 
            # outputs 가 0 ~ 1 사이 값으로 매핑 되지 않음
            train_pred = model(train_imgs)
            # calculate loss 
            train_loss = criterion(train_pred, train_labels)
            # backpropagation이 진행
            train_loss.backward()
            # model에서 update 가능한 gradient 값들을 모두 update 해줌
            optimizer.step()
            # 1 iter에서 나온 loss 값을 저장해줌. visualization 때문
            iter_train_loss.append(train_loss.cpu().item())

            print(f"epoch {epoch_idx}/{epoch_size}   iter {iter_idx}/{len(train_loader)}", end="\r")

        # gradient update가 안되게 함.
        # validation set은 학습이 되면 안되기 때문.
        with torch.no_grad():
            for iter_idx, (valid_imgs, valid_labels) in enumerate(valid_loader, 1) :
                # model을 evaluation 모드로 변경.
                model.eval()

                valid_imgs, valid_labels = valid_imgs.to(device=device, dtype=torch.float), valid_labels.to(device)

                valid_pred = model(valid_imgs)
                valid_loss = criterion(valid_pred, valid_labels)
                
                # 1 iter에서 나온 validation loss 값 저장
                iter_valid_loss.append(valid_loss.cpu().item())
                #  예측값이 가장 큰 index(0~6 값이 됨)와 label(6개 class 따라서 0 ~ 6)을 비교하여 정답인지 아닌지 알아냄.
                valid_pred_c = valid_pred.argmax(dim=-1)
                print(valid_pred_c)
                iter_valid_acc.extend((valid_pred_c==valid_labels).cpu().tolist())

                #  예측값과 label을 통한 f1 score 계산
                iter_f1_score = f1_score(y_true=valid_labels.cpu().numpy(), y_pred=valid_pred_c.cpu().numpy(), average="macro")
                iter_valid_f1.append(iter_f1_score)

                print(f"epoch {epoch_idx}/{epoch_size}   iter {iter_idx}/{len(valid_loader)}", end="\r")

        # 1 E에 포함된 iteration 동안 쌓여있는 loss 값들과  acc 값들의 평균을 구해줌.
        epoch_train_loss = np.mean(iter_train_loss)
        epoch_valid_loss = np.mean(iter_valid_loss)
        epoch_valid_acc = np.mean(iter_valid_acc) * 100
        epoch_valid_f1_score = np.mean(iter_valid_f1)

        result["train_loss"].append(epoch_train_loss)
        result["valid_loss"].append(epoch_valid_loss)
        result["valid_acc"].append(epoch_valid_acc)
        result["valid_f1"].append(epoch_valid_f1_score)
        
        # learing rate를 update 해줌.
        # optimizer.step()을 하고 난 뒤에 해줘야함.
        scheduler.step()

        print(
            f"[Epoch {epoch_idx}/{epoch_size}]"
            f"train loss : {epoch_train_loss:.4f} | "
            f"valid loss : {epoch_valid_loss:.4f} | "
            f"valid acc : {epoch_valid_acc:.2f} | "
            f"valid f1 score : {epoch_valid_f1_score:.4f}"
        )
        
        # 1 E 훈련이 끝날 때 f1 스코어가 best f1 스코어 보다 낫으면 early stoppinf stack을 증가 시킴.
        if epoch_valid_f1_score > best_f1 :
            best_f1 = epoch_valid_f1_score 
            best_model_state = model.state_dict()
            early_stop_count = 0
        else :
            early_stop_count += 1
        
        # early stopping stack이 가득차게 되면 break문으로 학습 중지.
        if early_stop_count == early_stop :
            print("early stopped.")
            break

    return result, best_model_state


# Dataset Parsing

In [7]:
import os
import cv2
import albumentations as A

from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset, DataLoader

class_encoder = {
    'dog' : 0,
    'elephant' : 1,
    'giraffe' : 2,
    'guitar' : 3,
    'horse' : 4,
    'house' : 5,
    'person' : 6
}

def img_gather_(img_path) :
    class_list = os.listdir(img_path)
    
    file_lists = []
    label_lists = []
    
    for class_name in class_list :
        file_list = os.listdir(os.path.join(img_path, class_name))
        # file_list에 img_path + class_name + file_list 경로를 다 더해서 넣어줌.
        file_list = list(map(lambda x: "/".join([img_path] + [class_name] + [x]), file_list))
        # 한 클래스에 포함된 image는 모두 같은 라벨이기 때문에
        # file_list 길이만큼 똑같은 라벨을 만들어 줌.
        label_list = [class_encoder[class_name]] * len(file_list)
        
        file_lists.extend(file_list)
        label_lists.extend(label_list)
        
    file_lists = np.array(file_lists)
    label_lists = np.array(label_lists)
    
    return file_lists, label_lists


class TrainDataset(Dataset) : 
    def __init__(self, file_lists, label_lists, transforms=None) :
        # 깊은 복사로 원본 list를 유지 시켜줌
        self.file_lists = file_lists.copy()
        self.label_lists = label_lists.copy()
        self.transforms = transforms
        
    def __getitem__(self, idx):
        img = cv2.imread(self.file_lists[idx], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # image에 trasform에 포함되어 있는 효과들을 적용
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        # Channel, W, H
        img = img.transpose(2, 0, 1)
        
        # self.label_lists에 직접 접근하지 않아도 indexing 할 수 있게 해주는 작업
        label = self.label_lists[idx]
        
        
        img = torch.tensor(img, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)
        
        return img, label

    def __len__(self):
        # image 개수와 label 개수가 맞을 때만 진행.
        assert len(self.file_lists) == len(self.label_lists)
        # instance에 len을 찾으면 이 함수가 발동
        return len(self.file_lists)
    
    
class TestDatasets(Dataset):
    def __init__(self, file_lists, transforms=None) :
        self.file_lists=file_lists.copy()
        self.transforms = transforms
    
    # test가 목적이므로 라벨은 의미 없음.
    def __getitem__(self, idx) :
        img = cv2.imread(self.file_lists[idx], cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if self.transforms :
            img = self.transforms(image=img)["image"]
            
        img = img.transpose(2, 0, 1)
        
        img = torch.tensor(img, dtype=torch.float)
        
        return img

    def __len__(self) :
        return len(self.file_lists)

In [8]:
# image에 적용할 효과들을 정의
train_transforms = A.Compose([
    A.Resize(224, 224), # 내가 추가
    A.Rotate(),
    A.HorizontalFlip(),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    A.Normalize()
])

valid_transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize()
])

In [9]:
# training set에 있는 image와 label을 가져옴
data_lists, data_labels = img_gather_("./data/train")

best_models = []

In [10]:
if k_fold_num == -1 :
    
    # training set에서 8:2 비율로 training set과 validation set으로 다시 나눔
    train_lists, valid_lists, train_labels, valid_labels = train_test_split(data_lists,
                                                                           data_labels,
                                                                           train_size=0.8,
                                                                           shuffle=True,
                                                                           random_state=random_seed,
                                                                           stratify=data_labels)
    
    train_dataset = TrainDataset(file_list=train_lists, label_lists=train_labels,
                                transforms = train_transforms)
    valid_dataset = TrainDataset(file_list=valid_lists, label_lists=valid_labels,
                                transforms = valid_transforms)
    
    # pytorch의 DataLoader를 사용하여 각 dataset 정의
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    
    data_loader = {"train_loader":train_loader, "valid_loader":valid_loader}
    
    print("No Fold training starts .... ")
    
    # 여기서 best_model은 model.state_dict()이므로 학습가능한 weight값만 포함된 상태임.
    # .pt 형태로 저장이 됨.
    # model.state_dict()를 사용하려면 model의 구조가 있어야 load 가능
    # model.load_state_dict()로 load 가능
    train_result, best_model = train(data_loader)
    
    best_models.append(best_model)
    
else:
    # StratifiedKFold는 각 라벨의 균형을 맞춰서 fold 를 할 때 데이터가 한 쪽으로 몰리는 현상을 방지해줌.
    skf = StratifiedKFold(n_splits=k_fold_num, random_state=random_seed, shuffle=True)
    
    print(f"{k_fold_num} fold training starts ... ")
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(data_lists, data_labels), 1) :
        print(f"- {fold_idx} fold -")
        train_lists, train_labels = data_lists[train_idx], data_labels[train_idx]
        valid_lists, valid_labels = data_lists[valid_idx], data_labels[valid_idx]
        
        train_dataset = TrainDataset(file_lists=train_lists, label_lists=train_labels,
                                    transforms=train_transforms)
        valid_dataset = TrainDataset(file_lists=valid_lists, label_lists=valid_labels,
                                    transforms=valid_transforms)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
        
        data_loader = {"train_loader":train_loader, "valid_loader":valid_loader}
        
        train_result, best_model = train(data_loader)
        
        best_models.append(best_model)

5 fold training starts ... 
- 1 fold -


c:\users\quhb2\anaconda3\envs\torch-1.9\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


TypeError: unsupported operand type(s) for /: 'list' and 'int'

### Inference

In [13]:
test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize()
])

test_files = os.listdir("./data/test/0")
test_files = sorted(test_files)
test_files = list(map(lambda x :"/".join(["./data/test/0", x]), test_files))

test_dataset = TestDatasets(file_lists=test_files, transforms=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
#!pip install pandas

In [15]:
import pandas as pd

answer_logits = []

model = timm.create_model(model_name, pretrained=True, num_classes=7).to(device=device)

for fold_idx, best_model in enumerate(best_models, 1) :
    model.load_state_dict(best_model)
    model.eval()
    
    fold_logits = []
    
    with torch.no_grad() :
        for iter_idx, test_imgs in enumerate(test_loader, 1) :
            test_imgs = test_imgs.to(device)
            
            test_pred = model(test_imgs)
            fold_logits.extend(test_pred.cpu().tolist())
            
            print(f"{fold_idx} fold  inference iteration {iter_idx}/{len(test_loader)}", end="\r")
            
    
    answer_logits.append(fold_logits)
    
answer_logits = np.mean(answer_logits, axis=0)
answer_value = np.argmax(answer_logits, axis=-1)

i = 0
while True :
    if not os.path.isfile(os.path.join("result", f"submission_{i}.csv")) :
        submission_path = os.path.join("result", f"submission_{i}.csv")
        break
    i += 1

print(submission_path)
submission = pd.read_csv("test_answer_sample_.csv", index_col=False)
submission["answer value"] = answer_value
submission["answer value"].to_csv(submission_path)
print("All done")

result\submission_0.csvtion 22/22
All done
